# Langchain!

## What is this?

The goal of this lab to introduce you to working with [Langchain](https://github.com/langchain-ai/langchain) for Python. [Langchain.js](https://github.com/langchain-ai/langchainjs) is also available for JavaScript/TypeScript, and (is supposed to be) very similar to the Python version. We don't need you to know every detail of the library (it's pretty huge), but you will need to study and understand the part(s) related to your eventual open source contribution. This notebook, alongside [the official documentation](https://python.langchain.com/v0.2/docs/how_to/), should give you a high-level overview.

## Install Langchain packages

In your Python environment, install `langchain-core`, `langchain` and `langchain-community`. The contents of each package are [briefly described here](https://python.langchain.com/v0.2/docs/concepts/#architecture), but in short, `langchain-community` has most of the third-party integrations—which is where you may want to consider contributing :)

Oh, and we'll need `requests` (just an HTTP client).

Run the next cell to install all that: <small><details><summary>(why v0.2 and not v0.3?)</summary>
<ul>
<li>v0.3 was released as I was writing this lab</li>
<li>the examples I'm linking to in the official documentation haven't been updated to work with v0.3</li>
</details></small>

In [ ]:
%pip install 'langchain-core>=0.2,<0.3' 'langchain>=0.2,<0.3' 'langchain-community>=0.2,<0.3' requests

## Finding a language model

Langchain supports two kinds of models: ["chat" models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which are fine-tuned for message-based conversations, and ["LLM" models](), which just take text ("once upon a time…") and spit out a completion ("…there was a"). Normally, you'd have to bring your own model to do anything with Langchain. But the author of this notebook already paid $5 to OpenAI during last year's D01. Might as well put it to use…

Run the next cell, and provide your UofT email + student number. This gives you access to a chat model you can experiment with. Feel free to mess around with this! But please do **not** spam or abuse this model, lest it burn a hole in your TA's pocket. (we will know who you are…)

If you want to run a serious Langchain application, or try some of the more advanced features like real-time streaming, use one of the [official chat models](https://python.langchain.com/v0.2/docs/integrations/chat/). These will require either a paid API key (if you use someone else's GPUs) or something like [Ollama](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/) (running on your own GPUs). (This space moves so fast that I'm not sure if Ollama is still the "new hotness," but it works.)

In [ ]:
import re

API_TOKEN = re.match('[^@]+(?=@)', input("Enter your UofT email: ")).group(0) + '-' + input("Enter your student number: ")
print(f'API token: {API_TOKEN}')

from typing import Any, Callable, Dict, List, Optional, Sequence, Type, Union

from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel, LanguageModelInput
from langchain_core.load import dumps, loads
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_tool
import requests


def fetch_from_relay(token, task, payload):
    res = requests.post('https://us-east5-cscd01-435202.cloudfunctions.net/chatmodel-relay', params={
        'token': token,
    }, json={
        'task': task,
        'payload': payload
    })

    if res.status_code != 200:
        headers = "\n".join((f"{k}: {v}" for k, v in res.headers.items()))
        raise Exception(f"Relay returned an error:\nHTTP {res.status_code}\n{headers}\n\n{res.text}")

    return loads(res.text)

class ChatCSCD01(BaseChatModel):
    """Langchain chat model for use in CSCD01.

    This is a network service for your personal use as a student.
    You must authenticate with the credentials provided by your TA.

    Feel free to use this service for testing.
    But do NOT spam or abuse it; we will know who you are.

    If you want to run a serious Langchain application, use one of the
    [official chat models](https://python.langchain.com/v0.2/docs/integrations/chat/).

    No need to understand this class, but for details on how it works, see
    - https://python.langchain.com/v0.2/docs/how_to/custom_chat_model/
    - https://python.langchain.com/v0.2/docs/how_to/serialization/
    """

    token: str
    """Authentication token."""

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        message_result = fetch_from_relay(self.token, 'chat', {
            'messages': [dumps(m) for m in messages],
            'stop': stop,
            'kwargs': kwargs
        })
        return ChatResult(generations=[ChatGeneration(message=message_result)])

    @property
    def _llm_type(self) -> str:
        return "chat-cscd01"

    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type, Callable, BaseTool]],
        *,
        strict: Optional[bool] = None,
        **kwargs: Any,
    ) -> Runnable[LanguageModelInput, BaseMessage]:
        return super().bind(tools=[convert_to_openai_tool(tool, strict=strict) for tool in tools], **kwargs)

class CSCD01Embeddings(Embeddings):
    """Langchain embedding model for use in CSCD01.

    This is a network service for your personal use as a student.
    You must authenticate with the credentials provided by your TA.

    Feel free to use this service for testing.
    But do NOT spam or abuse it; we will know who you are.

    If you want to run a serious Langchain application, use one of the
    [official embedding models](https://python.langchain.com/v0.2/docs/integrations/text_embedding/).
    """

    token: str
    """Authentication token."""

    def __init__(self, token: str):
        self.token = token

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return fetch_from_relay(self.token, 'embed_documents', texts)

    def embed_query(self, text: str) -> List[float]:
        return fetch_from_relay(self.token, 'embed_query', text)


model = ChatCSCD01(token=API_TOKEN)
embeddings = CSCD01Embeddings(token=API_TOKEN)

## Using the model

To use a chat model, it's as simple as:

In [ ]:
model.invoke('a man walked into a bar. (I need a punchline)')

But `.invoke` is not `ChatModel`-specific; it's part of the generic [`Runnable` interface](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). This is where the "chain" part of "Langchain" comes in.

## Chaining, prompts, and output parsers

Run this next cell:

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template('a {character} walked into a bar. (I need a punchline)')

chain = prompt | model | StrOutputParser()
chain

There are five `Runnable`s here: `prompt`, `model`, `StrOutputParser()`, `prompt | model` and `(prompt | model) | StrOutputParser()`, the last of which we called `chain`. The `|` ("pipe") operator behaves as you would expect: it streams output to input, left to right.

`.invoke` on a chain instance thus feeds the first `Runnable` and produces the result of the last `Runnable`:

In [ ]:
chain.invoke({'character': input('Who walked into a bar? ')})

Different `Runnable`s have different (heterogeneous) input and output. [`PromptTemplate`s](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) take a dictionary of values and produce a prompt message that a `ChatModel` can consume; [output parsers](https://python.langchain.com/v0.2/docs/concepts/#output-parsers) (attempt to) make structure from the chaos that is language model output.

Python functions can also be treated as `Runnable`s in a chain, as can other chains:

In [ ]:
from datetime import datetime

def show(output):
  print('show:', output)
  return output

extract_code = (
      PromptTemplate.from_template('extract just the code (nothing else!) from the following:\n{code}')
    | model
    | StrOutputParser()
    | (lambda x: {'code': x})
)

(
      PromptTemplate.from_template('write a Python program that prints the number of days left in the year')
    | show
    | model
    | StrOutputParser()

    | (lambda x: {'code': x})
    | show
    | extract_code
    | show

    | PromptTemplate.from_template('what does this program do?\n{code}')
    | model
    | StrOutputParser()
).invoke({'date': datetime.today().strftime('%Y-%m-%d')})

### Try it: Dummy data generation

By adapting [this example](https://python.langchain.com/v0.2/docs/how_to/output_parser_json/), use our chat model to generate place listings: name, rating, lat/lon Google Maps link, and anything else you like.

e.g. (Python output):
>     {'name': 'The Enchanted Coffeehouse',
>      'rating': 4.7,
>      'maps_link': 'https://www.google.com/maps/place/40.712776,-74.005974',
>      ...}

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure.
# TODO

# And a query intented to prompt a language model to populate the data structure.
# Tip: you may have to coax the agent into being a bit creative.
# TODO

# Set up a parser + inject instructions into the prompt template.
# TODO

prompt = PromptTemplate(
  # TODO
)
# If you're curious...
print("Format instructions:", parser.get_format_instructions())

chain = ... # TODO

# Generate some place listings
places = [chain.invoke(
  # TODO
) for _ in range(6)]
places

## Memory: document loaders, vector stores, and retrievers

Modern language models work by "[embedding](https://duckduckgo.com/?q=language+model+embedding)" text into a (very) high-dimensional vector space. Such a representation [efficiently encodes semantic similarity between tokens](https://towardsdatascience.com/text-embeddings-comprehensive-guide-afd97fce8fb5). "Vector stores" or "vector databases" are designed for efficient embedding storage and search.

Why should you care about this implementation detail? Here's one reason: say you have a large dataset you want your language model to reason with. At first, you might try to stuff all of it into the prompt string. Unfortunately, transformer models (like OpenAI's) have a "limited context window." In a way analogous to the limits of your own short-term memory, the model will tend to "forget" things that you told it a long time ago and never repeated. But, how do you, as a human, manage to reason with more information than you can hold in your short-term memory? You entrust it to your long-term memory (or your notebook, or the internet…). Vector stores perform a similar function for language models: at the model's request, you can retrieve information from a vector store, effectively augmenting its knowledge. This pattern is known as ["retrieval-augmented generation" (RAG)](https://towardsdatascience.com/rag-how-to-talk-to-your-data-eaf5469b83b0). (btw, don't take the human memory anaology too seriously.)

Chroma is a popular vector database:

In [ ]:
%pip install langchain-chroma

"Document loaders" are convenience utilities that break data down into individual documents that a model can ask to "see." There are [many built-in loaders](https://python.langchain.com/v0.2/docs/integrations/document_loaders/) that integrate with various data sources.

["Retrievers"](https://python.langchain.com/v0.2/docs/integrations/retrievers/) are `Runnable`s that query a data store. Vector stores are trivially retrievers ([`.as_retriever`](https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.as_retriever)).

### Try it: fake RAG

Read [this tutorial](https://python.langchain.com/v0.2/docs/tutorials/retrievers/) and/or [these](https://python.langchain.com/v0.2/docs/how_to/vectorstores/) [pages](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/). (You don't need to sign up for LangSmith or use any third-party API, unless you want to.) Then, by adapting [this example](https://python.langchain.com/v0.2/docs/tutorials/rag/), create an agent that can answer questions about the place listings you generated above.

This is "fake" RAG because the documents are just dumped into the prompt context. We'll fix that soon…

In [ ]:
from typing import Iterator
from langchain import hub
from langchain_core.document_loaders.base import BaseLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# TODO: Complete this document loader
class PlacesDocumentLoader(BaseLoader):
    """Simple document loader that yields the places generated above.

    See:
    - [BaseLoader](https://python.langchain.com/v0.2/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html)
    - [lazy_load](https://python.langchain.com/v0.2/api_reference/_modules/langchain_core/document_loaders/base.html#BaseLoader.lazy_load)
    - [Generators in Python](https://realpython.com/introduction-to-python-generators/)
    - [Document](https://python.langchain.com/v0.2/api_reference/core/documents/langchain_core.documents.base.Document.html#document)
    """

    def lazy_load(self) -> Iterator[Document]:
        pass # TODO


# Initialize your document loader, and load!
# For this example, we'll just get all documents at once.
loader = ... # TODO
docs = ... # TODO

# Split the documents into bite-sized chunks (that an LLM can handle).
splits = ... # TODO

# Dump the split documents into a vector store.
vectorstore = ... # TODO (hint: use `embeddings`, defined at the top of the notebook, as your embedding model)

# Retrieve and generate.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("can you give me a Google Maps link for a place to go for lunch?")

## Tools and agents

The final pieces of the puzzle for true RAG are [tools](https://python.langchain.com/v0.2/docs/concepts/#tools). In short, a tool is something a language model can "ask to use" with input of its choice. Modern ChatGPT has tools like web search and a Python runner (try it out!). In Langchain, tools are (you guessed it) `Runnable`s; but they also carry information that informs the model what the tool is and how to call it. Plain functions can be [wrapped with `@tool`](https://python.langchain.com/v0.2/api_reference/core/tools/langchain_core.tools.convert.tool.html#tool) to automatically generate this information.

When a model can take an action (such as using a tool), it's called an [agent](https://python.langchain.com/v0.2/docs/concepts/#agents). A common pattern for making agents with LLMs is [reason + act (ReAct)](https://python.langchain.com/v0.2/docs/concepts/#react-agents). Of course, to allow a model to take an external action and await feedback, we have be able to call both the tools and the model in a (possibly endless) loop; a simple pipeline won't suffice. We could certainly implement this ourselves, but no need: the Langchain team made another library just for this.

### LangGraph

LangGraph lets you build state machines for LLM-based agent control, with out-of-the-box support for the ReAct pattern. Notably, chat history is preserved between state transitions, and you can treat any Langchain tool as a state in the graph.

If you need a refresher on what "state machine" means, think back to "FSMs" from B58 or "PDAs" from B36, but where each state can access auxiliary memory. (If that sounds like a Turing machine—it is.)

In [ ]:
%pip install langgraph typing_extensions==4.12.2

### Try it: true RAG

Follow [this example](https://langchain-ai.github.io/langgraph/#example) to
- define a callable tool;
- define states and persistent memory for your agent (state machine);
- run your agent.

In [ ]:
from typing import Literal

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


# Define the tool for the agent to use
# TODO
# (hint: use your Chroma retriever)
# (hint: either use @tool with a good docstring, or follow [this guide](https://python.langchain.com/v0.2/docs/how_to/convert_runnable_to_tool/)
#        and pass a good description, so that the model knows what's up)


tools = # TODO
tool_node = (ToolNode(tools) | show)
model = model.bind_tools(tools)

# Transition function that runs after the model has returned a response
def after_agent_transition(state: MessagesState) -> Literal["tools", END]:
    # TODO (you can essentially copy the example code...)
    pass


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    print('messages:', messages)

    response = model.invoke(messages)
    print('response:', response)

    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
# TODO

# Set the entrypoint as `agent`
# This means that this node is the first one called
# TODO

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node.
    # TODO: your start state
    # Next, we pass in the function that will determine which node is called next.
    # TODO: your transition function
)

# We now add a normal edge from TODO to TODO.
# This means that after TODO is called, TODO node is called next.
workflow.add_edge(...)

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="please search for a place I can go for lunch. I cannot answer any further questions.")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content